In [ ]:
# Step 1: Baseline with Logistic Regression

In [ ]:
# 1. 导入必要的库
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import log_loss, classification_report, confusion_matrix
import string

# 2. 加载数据
train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

# 3. 特征工程：提取统计特征

def create_features(df):
    # 长度特征
    df['prompt_length'] = df['prompt'].str.len()
    df['response_a_length'] = df['response_a'].str.len()
    df['response_b_length'] = df['response_b'].str.len()
    # 单词数
    df['prompt_word_count'] = df['prompt'].str.split().str.len()
    df['response_a_word_count'] = df['response_a'].str.split().str.len()
    df['response_b_word_count'] = df['response_b'].str.split().str.len()
    # 标点数
    df['prompt_punc_count'] = df['prompt'].str.count(f'[{string.punctuation}]')
    df['response_a_punc_count'] = df['response_a'].str.count(f'[{string.punctuation}]')
    df['response_b_punc_count'] = df['response_b'].str.count(f'[{string.punctuation}]')
    # 差异特征
    df['response_length_diff'] = df['response_a_length'] - df['response_b_length']
    df['response_word_diff'] = df['response_a_word_count'] - df['response_b_word_count']
    return df

train = create_features(train)
test = create_features(test)

# 4. Label编码（可选：对model名称，但test不会用到）
le = LabelEncoder()
model_cols = []
for col in ['model_a', 'model_b']:
    train[f'{col}_enc'] = le.fit_transform(train[col])
    model_cols += [f'{col}_enc']

# 5. 构造标签
#   winner_model_a=1→类别0，winner_model_b=1→类别1，winner_tie=1→类别2
train['target'] = train[['winner_model_a', 'winner_model_b', 'winner_tie']].values.argmax(axis=1)

# 6. 选定全部数值特征
feature_cols = [
    'prompt_length', 'response_a_length', 'response_b_length',
    'prompt_word_count', 'response_a_word_count', 'response_b_word_count',
    'prompt_punc_count', 'response_a_punc_count', 'response_b_punc_count',
    'response_length_diff', 'response_word_diff'
    # 若希望，也可添加model_a_enc/model_b_enc
]

X = train[feature_cols]
y = train['target']

# 7. 数据拆分 + 标准化
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 8. 逻辑回归训练
clf = LogisticRegression(max_iter=1000, random_state=42, multi_class='ovr')
clf.fit(X_train_scaled, y_train)

# 9. 验证集效果
val_pred_proba = clf.predict_proba(X_val_scaled)
val_pred = clf.predict(X_val_scaled)
print('Validation Log Loss:', log_loss(y_val, val_pred_proba))
print('Classification Report:\n', classification_report(y_val, val_pred, digits=4))

# 10. 生成test特征、标准化并预测
X_test = test[feature_cols]
X_test_scaled = scaler.transform(X_test)
test_pred_proba = clf.predict_proba(X_test_scaled)

# 11. 生成Kaggle提交文件
submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': test_pred_proba[:,0],
    'winner_model_b': test_pred_proba[:,1],
    'winner_tie': test_pred_proba[:,2],
})
submission.to_csv('submission_baseline.csv', index=False)
print(submission.head())


In [ ]:
#Step 2: Embedding-based model

In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

# 用本地模型路径加载（路径需和你 Add Input 名称一致）
model = SentenceTransformer('/kaggle/input/minilm-l12-v2-local/other/default/1/minilm_l12_v2_local')

# 拼接文本
def concat_text(df):
    return (
        df['prompt'].astype(str) + ' ' + df['response_a'].astype(str),
        df['prompt'].astype(str) + ' ' + df['response_b'].astype(str)
    )
train_a, train_b = concat_text(train)
test_a, test_b = concat_text(test)

emb_a = model.encode(train_a.tolist(), batch_size=32, show_progress_bar=True)
emb_b = model.encode(train_b.tolist(), batch_size=32, show_progress_bar=True)
X = np.hstack([emb_a, emb_b, emb_a - emb_b])
y = train[['winner_model_a','winner_model_b','winner_tie']].values.argmax(axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
print('Val Log Loss:', log_loss(y_val, clf.predict_proba(X_val)))

emb_a_test = model.encode(test_a.tolist(), batch_size=32, show_progress_bar=True)
emb_b_test = model.encode(test_b.tolist(), batch_size=32, show_progress_bar=True)
X_test = np.hstack([emb_a_test, emb_b_test, emb_a_test - emb_b_test])
proba = clf.predict_proba(X_test)

submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': proba[:,0],
    'winner_model_b': proba[:,1],
    'winner_tie': proba[:,2]
})
submission.to_csv('submission_emb.csv', index=False)


Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Val Log Loss: 1.0668766656893598


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]